In [211]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns




# 1. Traitement des données 

Importation du jeu de données et division en données classe et caractéristique

In [223]:
dt = pd.read_csv("/Users/mallorylecorre/Desktop/Dm_Ia/cars_data-class.csv")
print(dt.head())

X = dt.drop('Impact_Environnement', axis=1)
y = dt['Impact_Environnement']

   Taille_Auto   Poids_Auto  Acceleration  Annee_Fabrication    Kilometrage  \
0     3.790317  8869.946374      2.038688               2006  135542.067972   
1     3.375892  1126.174079      2.067411               2010   33710.551662   
2     4.844971  6100.019066      4.839676               2016  112977.279109   
3     2.316172  5771.509867      1.202574               2014   50139.727635   
4     2.618825  5676.666211      5.470806               2009  191188.750210   

   Nombre_Portes  Vitesse_Max  Prix_Assurance  Categorie_Voiture    Decote  \
0              5   195.289066      184.125577                  1  0.104588   
1              5   249.979656      858.935088                  3  0.346064   
2              5   235.763320      302.138838                  4  0.394582   
3              4   216.802020     1298.671966                  5  0.422048   
4              4   234.004505      702.626517                  2  0.279959   

   Impact_Environnement  
0                     1  
1   

In [224]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [225]:
dt['Impact_Environnement'].value_counts()

1    537
3    385
2     42
0     36
Name: Impact_Environnement, dtype: int64

On peut voir que le jeu de données est déséquilibrée avec une sur-représentation de la classe 1.

On équilibre les données en sur-échantillonant avec SMOTE

In [226]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


On vérifie que le rééquillibrage a bien fonctionné

In [227]:
print("Nouvelle distribution :")
print(pd.Series(y_train).value_counts())

Nouvelle distribution :
1    428
3    428
0    428
2    428
Name: Impact_Environnement, dtype: int64


Les données sont équilibrées

# Création du modèle 

In [228]:
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [229]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(4, activation='softmax'))

compilation du modèle en tuilisant une fonction de perte qui correspond à une classification multi-class

In [230]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Entrainement du modèle 

In [239]:
model.fit(X_train, y_train, epochs=50, batch_size=40, validation_split=0.2)


Epoch 1/50
35/35 [==============================] - 0s 2ms/step - loss: 1.3475 - accuracy: 0.3017 - val_loss: 1.7401 - val_accuracy: 0.0000e+00
Epoch 2/50
35/35 [==============================] - 0s 964us/step - loss: 1.3468 - accuracy: 0.3258 - val_loss: 1.7420 - val_accuracy: 0.0000e+00
Epoch 3/50
35/35 [==============================] - 0s 977us/step - loss: 1.3484 - accuracy: 0.3134 - val_loss: 1.7702 - val_accuracy: 0.0000e+00
Epoch 4/50
35/35 [==============================] - 0s 970us/step - loss: 1.3465 - accuracy: 0.3141 - val_loss: 1.7663 - val_accuracy: 0.0000e+00
Epoch 5/50
35/35 [==============================] - 0s 993us/step - loss: 1.3478 - accuracy: 0.3046 - val_loss: 1.7558 - val_accuracy: 0.0000e+00
Epoch 6/50
35/35 [==============================] - 0s 1ms/step - loss: 1.3472 - accuracy: 0.3097 - val_loss: 1.7541 - val_accuracy: 0.0000e+00
Epoch 7/50
35/35 [==============================] - 0s 2ms/step - loss: 1.3491 - accuracy: 0.3112 - val_loss: 1.7648 - val_accur

# Evalutation du modèle

In [240]:
eval_results = model.evaluate(X_test, y_test_categorical)

# Récupérer l'accuracy depuis les résultats de l'évaluation
accuracy = eval_results[1]

# Afficher l'accuracy
print("Accuracy:", accuracy)

7/7 [==============================] - 0s 691us/step - loss: 1.3084 - accuracy: 0.5350
Accuracy: 0.5350000262260437


affichaqge des différents score et de la matrice de confusion